In [1]:
print("hello")

hello


# データセットダウンロード

In [5]:
!pip install GEOparse

  Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_arm64.whl (5.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 5.3 MB/s  0:00:02 eta 0:00:01
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached requests-2.32.5-py3-none-any

In [6]:
import GEOparse
import pandas as pd

# 例として GSE60670（神経損傷後の脊髄ミクログリア）を取得
# 初回は時間がかかる＆ネット必須
gse_id = "GSE60670"
gse = GEOparse.get_GEO(geo=gse_id, destdir="./geo_data")

# メタ情報をざっと確認
print("Title :", gse.metadata.get("title"))
print("Samples:", len(gse.gsms))
print("Platforms:", list(gse.gpls.keys()))

# 1つサンプルを見てみる
first_gsm = next(iter(gse.gsms.values()))
print("\n[Sample ID]", first_gsm.name)
print("[Sample metadata keys]:", list(first_gsm.metadata.keys())[:20])

# 発現テーブル（遺伝子 x サンプル）の作成
# 多くの array / RNA-seq 系 GEO はこの書き方で取れます
expr_df = gse.pivot_samples('VALUE')

print(expr_df.shape)
expr_df.head()

28-Nov-2025 18:50:07 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE60nnn/GSE60670/soft/GSE60670_family.soft.gz to ./geo_data/GSE60670_family.soft.gz
100%|██████████| 17.4M/17.4M [00:07<00:00, 2.44MB/s]
28-Nov-2025 18:50:23 DEBUG downloader - Size validation passed
28-Nov-2025 18:50:23 DEBUG downloader - Moving /var/folders/qm/spddkd_11h33syxzdcnk9cf00000gn/T/tmp5oxtyvgv to /Users/kakuayato/Documents/GitHub/PharmFoundation/geo_data/GSE60670_family.soft.gz
28-Nov-2025 18:50:23 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE60nnn/GSE60670/soft/GSE60670_family.soft.gz
28-Nov-2025 18:50:23 INFO GEOparse - Parsing ./geo_data/GSE60670_family.soft.gz: 
28-Nov-2025 18:50:23 DEBUG GEOparse - DATABASE: GeoMiame
28-Nov-2025 18:50:23 DEBUG GEOparse - SERIES: GSE60670
28-Nov-2025 18:50:23 DEBUG GEOparse - PLATFORM: GPL6246
28-Nov-2025 18:50:24 DEBUG GEOparse - SAMPLE: GSM1484360
28-Nov-2025 18:50:24 DEBUG GEOparse - SAMPLE: GSM1484361
28-No

Title : ['High-resolution transcriptome analysis reveals neuropathic pain gene-expression signatures in spinal microglia after nerve injury']
Samples: 23
Platforms: ['GPL6246']

[Sample ID] GSM1484360
[Sample metadata keys]: ['title', 'geo_accession', 'status', 'submission_date', 'last_update_date', 'type', 'channel_count', 'source_name_ch1', 'organism_ch1', 'taxid_ch1', 'characteristics_ch1', 'molecule_ch1', 'extract_protocol_ch1', 'label_ch1', 'label_protocol_ch1', 'hyb_protocol', 'scan_protocol', 'data_processing', 'platform_id', 'contact_name']
(34760, 23)


name,GSM1484360,GSM1484361,GSM1484362,GSM1484363,GSM1484364,GSM1484365,GSM1484366,GSM1484367,GSM1484368,GSM1484369,...,GSM1484373,GSM1484374,GSM1484375,GSM1484376,GSM1484377,GSM1484378,GSM1484379,GSM1484380,GSM1484381,GSM1484382
ID_REF,,,,,,,,,,,,,,,,,,,,,
10338001,13.297266,13.814771,13.368966,13.625138,13.650088,13.757673,13.687413,13.574463,13.478501,13.477558,...,14.100938,13.336863,14.190183,13.230050,13.447355,13.115552,13.659530,13.277454,14.055461,13.418335
10338003,12.854826,13.505932,12.979227,13.327938,13.366751,13.458456,13.362361,13.278185,13.154560,13.093526,...,13.869679,13.115258,13.902486,12.889436,13.101658,12.968892,13.363629,13.066771,13.762874,13.135692
10338004,10.260992,11.036145,10.682868,10.780150,10.891785,10.594008,10.518250,10.581445,10.846925,10.600024,...,11.614107,11.093898,12.135710,10.346013,11.173720,11.662076,10.922440,10.828737,11.875394,10.306455
10338017,13.429894,14.089884,13.744737,14.018052,14.076960,14.126879,14.135867,13.958816,13.753550,13.987237,...,14.230537,13.723422,14.305796,13.681901,13.863663,13.796180,14.063768,13.614221,14.263738,13.774395
10338025,9.946308,10.444366,10.573800,10.137686,10.097755,10.380183,10.140556,10.155301,10.167941,9.954050,...,11.240420,10.956082,11.752362,9.740592,10.910287,11.512533,10.391802,10.485938,11.492515,10.071201
